# Clone and install specified Python module

In [1]:
COLAB_FOLDER = 'AIMS' # change as needed
GITHUB_USER  = 'soot-bit'
GITHUB_REPO  = 'pinn4bhoc'
GITHUB_FOLDERS  = ['pinn4bhoc']

In [2]:
CHECK = "\u2705"
FAIL  = "\u274C"
WARN  = "\u26A0"
# --------------------------------------------------------------------
def git_clone(user, repo, folders):
    import os
    import subprocess

    url      = f'https://github.com/{user}/{repo}'
    raw_url  = url.replace('github.com', 'raw.githubusercontent.com')
    raw_url += '/refs/heads/main/'
    toml_file= 'pyproject.toml'

    # 1. Delete local repo folder if it exists
    print(f'\t1. delete old repo: {repo}')
    result = subprocess.run(
        ["rm", "-rf", repo],
        capture_output=True)
    if result.returncode != 0:
        print(f"{FAIL} Deletion of {repo} failed:",
              result.stderr.strip())
        return result.returncode

    # 2. Clone repo <repo> with minimal history
    print(f'\t2. clone repo: {repo}')
    result = subprocess.run(
        ["git", "clone",
         "--depth", "1",
         "--filter=blob:none",
         "--sparse", url],
        capture_output=True,
        text=True)
    if result.returncode == 0:
        print(result.stdout.strip())
    else:
        print(f"{FAIL} Clone of {repo} failed:", result.stderr.strip())
        return result.returncode

    # 3. Checkout specified folders
    print(f'\t3. cd to repo folder: {repo}')
    os.chdir(repo) # move to repo folder
    for folder in folders:
        print(f'\t4. check out folder: {folder}')
        result = subprocess.run(
        	["git", "sparse-checkout",
          	"set", folder],
        	capture_output=True,
        	text=True)
        if result.returncode == 0:
        	print(result.stdout.strip())
        else:
            print(f"{FAIL} Sparse checkout of {dirname} failed:",
                  result.stderr.strip())
            os.chdir("..")
            return result.returncode

    # 5. wget toml_file
    print(f'\t5. wget {toml_file}')
    url = raw_url + toml_file
    result = subprocess.run(
        ["wget", "-q", url, "-O", toml_file],
        capture_output=True, text=True)
    if result.returncode != 0:
        print(f"{FAIL} wget of {toml_file} failed:", result.stderr.strip())
        return result.returncode

    # 6. pip install -e . module
    print(f'\t6. pip install -e .\n')
    result = subprocess.run(
        ["pip", "install", "-e", '.'],
        capture_output=True,
        text=True)
    if result.returncode == 0:
        print(result.stdout.strip())
    else:
        print(f"{FAIL} pip install -e . failed: ",
              result.stderr.strip())
        os.chdir("..")
        return result.returncode

    # 5. Success! remember to go back to code folder
    print(f'\n{CHECK} Installation of {repo} successful')
    os.chdir("..")
    return 0

In [3]:
try:
    from google.colab import drive
    drive.mount('/content/gdrive')
    print('\nGoogle Drive mounted\n')
    %cd /content/gdrive/MyDrive/{COLAB_FOLDER}
    %pwd

    print('Running on Colab')
    IN_COLAB = True
except:
    print('Running locally')
    IN_COLAB = False

if IN_COLAB:
    git_clone(GITHUB_USER, GITHUB_REPO, GITHUB_FOLDERS)

Running locally
